In [1]:
## 3/ spustit trénování pouze pro třídy 8 a 6 (klasifikační problém, kdy se snažíme rozeznat osmičku od šestky), kolik vzorků z každé třídy používáme? jaká je úspěšnost?

In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

num_classes = 10
input_shape = (28, 28, 1)

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

#pomocne listy pro ukladani dat
x_train_list, y_train_list, x_test_list, y_test_list = [], [], [], []
counter = [0]*4

for i, y in enumerate(y_train):
    if y == 6:
        y_train_list.append(y)
        x_train_list.append(x_train[i])
        counter[0]+=1
    elif y == 8:
        y_train_list.append(y)
        x_train_list.append(x_train[i])
        counter[1]+=1

for i, y in enumerate(y_test):
    if y == 6:
        y_test_list.append(y)
        x_test_list.append(x_test[i])
        counter[2]+=1
    elif y == 8:
        y_test_list.append(y)
        x_test_list.append(x_test[i])
        counter[3]+=1
        
x_train, y_train, x_test, y_test = np.array(x_train_list), np.array(y_train_list), np.array(x_test_list), np.array(y_test_list)

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print('{} train samples ({}x 6, {}x 8)'.format(x_train.shape[0], counter[0], counter[1]))
print('{} test samples ({}x 6, {}x 8)'.format(x_test.shape[0], counter[2], counter[3]))


y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


11769 train samples (5918x 6, 5851x 8)
1932 test samples (958x 6, 974x 8)


[kitt] Hezky vyřešeno. U counter by šlo alternativně využít i Dictionary, třeba: counter = {'train_6': 0, 'train_8': 0, 'test_6': 0, 'test_8': 0} a poté se přistupuje: counter['train_6'] += 1 atd. (kolem "+=" bych psal mezery... :-) Jinak pecka.

In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [4]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/15
83/83 [==============================] - 3s 28ms/step - loss: 0.3339 - accuracy: 0.8754 - val_loss: 0.0263 - val_accuracy: 0.9932
Epoch 2/15
83/83 [==============================] - 2s 27ms/step - loss: 0.0409 - accuracy: 0.9858 - val_loss: 0.0171 - val_accuracy: 0.9949
Epoch 3/15
83/83 [==============================] - 2s 27ms/step - loss: 0.0262 - accuracy: 0.9909 - val_loss: 0.0128 - val_accuracy: 0.9958
Epoch 4/15
83/83 [==============================] - 2s 27ms/step - loss: 0.0204 - accuracy: 0.9928 - val_loss: 0.0099 - val_accuracy: 0.9966
Epoch 5/15
83/83 [==============================] - 2s 27ms/step - loss: 0.0170 - accuracy: 0.9941 - val_loss: 0.0116 - val_accuracy: 0.9966
Epoch 6/15
83/83 [==============================] - 2s 27ms/step - loss: 0.0156 - accuracy: 0.9950 - val_loss: 0.0121 - val_accuracy: 0.9966
Epoch 7/15
83/83 [==============================] - 2s 27ms/step - loss: 0.0116 - accuracy: 0.9968 - val_loss: 0.0092 - val_accuracy: 0.9966
Epoch 8/15
83

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
#uspesnost
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.006161822937428951
Test accuracy: 0.998964786529541
